<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/Colab_Script_Datan_Esik%C3%A4sittely_ja_Tallennus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
Colab Script: Data Preprocessing and Saving

Loads Kallio ozone and Kaisaniemi weather data, merges them,
resamples to hourly frequency, interpolates missing values,
and saves the final processed DataFrame to a Parquet file.
"""

# Kirjastojen tuonti
import pandas as pd
import numpy as np
import io
import requests
import csv
import traceback
import re # Tarvitaan sarakkeiden nimien puhdistukseen, jos käytetään vanhaa funktiota

# Varmista, että tarvittava kirjasto Parquet-tiedostoille on asennettu
# Voit poistaa kommentin alta ajaaksesi asennuksen Colabissa tarvittaessa
# !pip install pyarrow -q
# tai
# !pip install fastparquet -q

print("--- Data Preprocessing and Saving Script ---")

# --- Funktiot datan lataamiseen ja peruspuhdistukseen ---
# (Sama kuin aiemmissa skripteissä, varmistetaan että se on tässä)
def load_and_clean_data(raw_url, data_type='weather', cols_to_keep=None):
    """Lataa ja esikäsittelee datan, pitäen vain tarvittavat sarakkeet."""
    print(f"\nLadataan {data_type} dataa: {raw_url}")
    df_local = None
    try:
        response = requests.get(raw_url)
        response.raise_for_status()
        encoding = 'utf-8' if data_type == 'weather' else 'iso-8859-1'
        print(f"Lukeminen CSV:stä (encoding={encoding})...")
        csv_content = io.StringIO(response.content.decode(encoding))

        if data_type == 'ozone':
             column_names_ozone = ["Havaintoasema", "Vuosi", "Kuukausi", "Päivä", "Aika [Paikallinen aika]", "Otsoni [µg/m³]"]
             df_local = pd.read_csv(
                  csv_content, sep=',', decimal=',', skiprows=1, header=None,
                  names=column_names_ozone, quoting=csv.QUOTE_NONNUMERIC, low_memory=False
             )
             target_col = "Otsoni [µg/m³]"
             if target_col in df_local.columns:
                  df_local[target_col] = pd.to_numeric(df_local[target_col], errors='coerce')
             else: raise ValueError(f"Otsonisarake '{target_col}' ei löytynyt.")

        elif data_type == 'weather':
            df_local = pd.read_csv(csv_content, sep=',', decimal='.', low_memory=False)
            df_local.columns = df_local.columns.str.strip()
            # Muunna kaikki sarakkeet numeroiksi, joita aiotaan säilyttää
            weather_cols_to_convert = ['Lämpötilan keskiarvo [°C]', 'Keskituulen nopeus [m/s]', 'Tuulen suunnan keskiarvo [°]', 'Ilmanpaineen keskiarvo [hPa]']
            for col in weather_cols_to_convert:
                 if col in df_local.columns:
                      # Varmista, että yritetään muuntaa vain ne, jotka on pyydetty säilyttämään
                      if cols_to_keep is None or col in cols_to_keep:
                           df_local[col] = pd.to_numeric(df_local[col], errors='coerce')

        else: raise ValueError(f"Tuntematon data_type: {data_type}")

        # Luo Timestamp
        year_col, month_col, day_col, time_col = 'Vuosi', 'Kuukausi', 'Päivä', 'Aika [Paikallinen aika]'
        required_dt_cols = [year_col, month_col, day_col, time_col]
        if not all(col in df_local.columns for col in required_dt_cols):
            raise ValueError(f"Aikaleiman luontiin vaadittavia sarakkeita puuttuu: {required_dt_cols}")

        df_local[year_col] = pd.to_numeric(df_local[year_col], errors='coerce').astype('Int64')
        df_local[month_col] = pd.to_numeric(df_local[month_col], errors='coerce').astype('Int64')
        df_local[day_col] = pd.to_numeric(df_local[day_col], errors='coerce').astype('Int64')
        if df_local[required_dt_cols].isnull().any().any():
            df_local.dropna(subset=required_dt_cols, inplace=True)

        df_local[year_col] = df_local[year_col].astype(str)
        df_local[month_col] = df_local[month_col].astype(str).str.zfill(2)
        df_local[day_col] = df_local[day_col].astype(str).str.zfill(2)
        df_local[time_col] = df_local[time_col].astype(str)
        time_str = df_local[time_col].str.replace('24:00', '00:00', regex=False)
        datetime_str = df_local[year_col] + '-' + df_local[month_col] + '-' + df_local[day_col] + ' ' + time_str
        df_local['Timestamp'] = pd.to_datetime(datetime_str, format='%Y-%m-%d %H:%M', errors='coerce')
        df_local.loc[df_local[time_col] == '24:00', 'Timestamp'] += pd.Timedelta(days=1)

        # Valitse vain pyydetyt sarakkeet + Timestamp
        cols_to_select = ['Timestamp'] + (cols_to_keep if cols_to_keep else [])
        missing_cols = [col for col in cols_to_select if col not in df_local.columns]
        if missing_cols:
             raise ValueError(f"Vaadittuja sarakkeita puuttuu datasta: {missing_cols}")
        df_local = df_local[cols_to_select].copy()

        # Poista rivit, joissa Timestamp tai jokin säilytettävä sarake on NaN
        df_local.dropna(subset=['Timestamp'] + (cols_to_keep if cols_to_keep else []), inplace=True)

        # Aseta indeksi ja poista duplikaatit
        df_local.set_index('Timestamp', inplace=True)
        df_local.sort_index(inplace=True)
        duplicates = df_local.index.duplicated(keep='first')
        if duplicates.sum() > 0:
            df_local = df_local[~duplicates]

        print(f"Datan ({data_type}) käsittely valmis, muoto: {df_local.shape}")
        if df_local.empty: print(f"VAROITUS: {data_type} DataFrame on tyhjä!")
        return df_local

    except requests.exceptions.RequestException as e:
        print(f"Virhe datan haussa URL:sta ({data_type}): {e}")
        return None
    except Exception as e:
        print(f"Virhe datan käsittelyssä ({data_type}): {e}")
        traceback.print_exc()
        return None

# --- Pääohjelma ---
try:
    # 1. Lataa datat
    ozone_url = "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kallio%202_%201.4.2020%20-%201.4.2025_f5d0d5ac-9f7d-4833-a70b-c1afe4dc935a.csv"
    weather_url = "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kaisaniemi_%201.4.2020%20-%201.4.2025_d5590617-bf91-46c7-96f4-1fb70892265d.csv"
    o3_col = "Otsoni [µg/m³]"
    # Otetaan kaikki relevantit säämuuttujat mukaan prosessoituun dataan
    weather_cols = ['Lämpötilan keskiarvo [°C]', 'Keskituulen nopeus [m/s]', 'Ilmanpaineen keskiarvo [hPa]', 'Tuulen suunnan keskiarvo [°]']

    df_ozone = load_and_clean_data(ozone_url, data_type='ozone', cols_to_keep=[o3_col])
    df_weather = load_and_clean_data(weather_url, data_type='weather', cols_to_keep=weather_cols)

    # 2. Yhdistä ja Resample
    df_processed = None # Alustus
    if df_ozone is not None and not df_ozone.empty and df_weather is not None and not df_weather.empty:
        print("\nYhdistetään otsoni- ja säädata...")
        df_merged_raw = pd.merge(df_ozone, df_weather, left_index=True, right_index=True, how='inner')
        df_merged_raw.dropna(inplace=True) # Poista rivit joissa NaN jossain sarakkeessa
        print(f"Yhdistetty data, rivejä NaN-poiston jälkeen: {len(df_merged_raw)}")

        if not df_merged_raw.empty:
            print("Uudelleenotanta ('resample') tunneittaiseen taajuuteen...")
            df_processed = df_merged_raw.resample('h').mean()
            print(f"Rivejä resamplen jälkeen: {len(df_processed)}")

            # Käsittele resamplen luomat NaN-arvot
            nan_after_resample = df_processed.isnull().sum().sum()
            if nan_after_resample > 0:
                print(f"Löytyi {nan_after_resample} NaN-arvoa resamplen jälkeen. Interpoloidaan (time)...")
                df_processed.interpolate(method='time', inplace=True)
                # Täytä mahdolliset alku-/loppu-NaN:t
                remaining_nan = df_processed.isnull().sum().sum()
                if remaining_nan > 0:
                     print(f"Varoitus: Jäljellä {remaining_nan} NaN-arvoa interpoloinnin jälkeen. Täytetään (ffill/bfill)...")
                     df_processed.fillna(method='ffill', inplace=True)
                     df_processed.fillna(method='bfill', inplace=True)
            # Viimeinen tarkistus ja mahdollisten jäljelle jääneiden NaN-rivien poisto
            if df_processed.isnull().any().any():
                 print("VAROITUS: NaN-arvoja jäi datan käsittelyn jälkeen! Poistetaan rivit.")
                 df_processed.dropna(inplace=True)

            if df_processed.empty:
                print("Virhe: Data tyhjä resamplen/NaN-käsittelyn jälkeen.")
                df_processed = None
            else:
                print("Datan yhdistäminen ja resample onnistui.")
                print("\nLopullisen prosessoidun datan info:")
                df_processed.info()
                print("\nLopullisen prosessoidun datan 5 ensimmäistä riviä:")
                print(df_processed.head())
        else:
            print("Virhe: Yhdistetty data tyhjä ennen resamplea.")
            df_processed = None

    # 3. Tallenna prosessoitu data
    if df_processed is not None and not df_processed.empty:
        save_filename = 'processed_hourly_ozone_weather_data.parquet'
        # save_filename_csv = 'processed_hourly_ozone_weather_data.csv' # Vaihtoehtoinen CSV

        print(f"\nTallennetaan prosessoitu data tiedostoon: {save_filename}")
        try:
            # Käytä index=True säilyttääksesi DatetimeIndexin
            df_processed.to_parquet(save_filename, index=True)
            print("Tallennus Parquet-muotoon onnistui.")
            # Voit tallentaa myös CSV:ksi:
            # df_processed.to_csv(save_filename_csv, index=True)
            # print("Tallennus CSV-muotoon onnistui.")

            print("\nVoit nyt ladata tämän tiedoston Colabista tai käyttää sitä suoraan")
            print("seuraavissa mallinnusscripteissä lukemalla sen esim:")
            print("`df = pd.read_parquet('processed_hourly_ozone_weather_data.parquet')`")

        except ImportError:
             print("\nVirhe: Parquet-tallennus vaatii 'pyarrow' tai 'fastparquet' kirjaston.")
             print("Asenna se Colabissa komennolla: !pip install pyarrow")
        except Exception as e_save:
            print(f"Virhe tiedoston tallennuksessa: {e_save}")
            traceback.print_exc()
    else:
        print("\nProsessoitu data on tyhjä tai sitä ei luotu, mitään ei tallennettu.")


except Exception as e_main:
    print(f"\nSkriptin suorituksessa tapahtui odottamaton virhe: {e_main}")
    traceback.print_exc()

print("\n--- Esikäsittelyskripti päättyi ---")

--- Data Preprocessing and Saving Script ---

Ladataan ozone dataa: https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kallio%202_%201.4.2020%20-%201.4.2025_f5d0d5ac-9f7d-4833-a70b-c1afe4dc935a.csv
Lukeminen CSV:stä (encoding=iso-8859-1)...
Datan (ozone) käsittely valmis, muoto: (43180, 1)

Ladataan weather dataa: https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kaisaniemi_%201.4.2020%20-%201.4.2025_d5590617-bf91-46c7-96f4-1fb70892265d.csv
Lukeminen CSV:stä (encoding=utf-8)...
Datan (weather) käsittely valmis, muoto: (43455, 4)

Yhdistetään otsoni- ja säädata...
Yhdistetty data, rivejä NaN-poiston jälkeen: 42795
Uudelleenotanta ('resample') tunneittaiseen taajuuteen...
Rivejä resamplen jälkeen: 43848
Löytyi 5265 NaN-arvoa resamplen jälkeen. Interpoloidaan (time)...
Datan yhdistäminen ja resample onnistui.

Lopullisen prosessoidun datan info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 438